# 📊 PyPortfolio - Otimização de Portfólio v2.3**Markowitz, Fronteira Eficiente, Monte Carlo e Sharpe vs CDI**---### Changelog v2.3- ✅ Suporte a leitura de retornos e pesos do Excel (v7)- ✅ Hierarquia de fontes implementada (Excel → yfinance → DEFAULT_UNIVERSE)- ✅ Carteira atual exibida como estrela (⭐) no gráfico- ✅ Média geométrica para retorno esperado (opção)- ✅ Validação de matriz PSD com correção automática- ✅ Filtros MIN_OBS e MIN_OVERLAP- ✅ Relatório de mapeamento e cobertura de pesos- ✅ Código organizado em seções com funções utilitárias- ✅ Configurações centralizadas no topo

---## 🔧 BLOCO 0 - CONFIGURAÇÕES GLOBAISCentralize aqui todas as variáveis de configuração do projeto.

In [ ]:
# =============================================================================# CONFIGURAÇÕES GLOBAIS - EDITE AQUI# =============================================================================# --- Caminhos de Arquivos ---EXCEL_PATH = "1 - Dados/1 - Rentabilidade atual/Rendimentos_Mensais_Ativos_v7.0.xlsx"# --- Período de Análise (usado se baixar de yfinance) ---DATA_INICIO = "2020-01-01"DATA_FIM = "2026-01-01"# --- Parâmetros de Qualidade de Dados ---MIN_OBS = 12            # Mínimo de meses por ativo para entrar na otimizaçãoMIN_OVERLAP = 12        # Mínimo de meses em comum entre pares para covariância confiávelVOL_FLOOR = 0.001       # Piso de volatilidade mensal (0.1%) para ativos "suaves"# --- Taxa Livre de Risco ---RF_MODO = "atual"       # "atual" | "media_periodo" | "manual"RF_MANUAL = 0.1150      # 11.5% a.a. (usado se RF_MODO="manual")RF_FALLBACK = 0.10      # 10% a.a. se tudo falhar# --- Parâmetros de Otimização ---PESO_MAX_ATIVO = 1.0    # Máximo 100% em um único ativoPESO_MIN_ATIVO = 0.0    # Mínimo 0% (long-only)# --- Simulação Monte Carlo ---NUM_PORTFOLIOS = 50000  # Quantidade de carteiras simuladasRANDOM_SEED = 42        # Semente para reprodutibilidade# --- Retorno: Média Aritmética vs Geométrica ---USAR_MEDIA_GEOMETRICA = True  # True = mais conservador e realista# --- Calendário para Anualização ---MESES_ANO = 12DIAS_UTEIS_ANO = 252USAR_CALENDARIO_DIARIO = False  # False para Excel mensal# --- Universo Padrão (fallback) ---DEFAULT_UNIVERSE = [    "PETR4.SA", "ITUB4.SA", "VALE3.SA", "BBAS3.SA",    "GOOGL", "NVDA", "META", "AMZN", "VOO",    "BTC-USD", "SOL-USD"]# --- Visualização ---TOP_N_PESOS = 15print("✅ Configurações carregadas!")print(f"   Excel: {EXCEL_PATH}")print(f"   MIN_OBS: {MIN_OBS} meses | Média geométrica: {USAR_MEDIA_GEOMETRICA}")

---## 📦 BLOCO 1 - BIBLIOTECAS E FUNÇÕES UTILITÁRIAS

In [ ]:
# =============================================================================# IMPORTS# =============================================================================import pandas as pdimport numpy as npfrom scipy.optimize import minimizefrom scipy.linalg import eighimport plotly.graph_objects as goimport warningsfrom pathlib import Pathfrom difflib import SequenceMatcherimport refrom typing import Optional, Dict, Tuple, Listfrom IPython.display import displaytry:    import yfinance as yf    YFINANCE_DISPONIVEL = Trueexcept ImportError:    YFINANCE_DISPONIVEL = False    print("⚠️ yfinance não instalado. Apenas fonte Excel disponível.")warnings.filterwarnings('ignore', category=FutureWarning)pd.set_option('display.max_columns', 20)print("✅ Bibliotecas importadas!")

In [ ]:
# =============================================================================# FUNÇÕES - LEITURA E PROCESSAMENTO DE DADOS# =============================================================================def normalizar_nome(nome: str) -> str:    """Normaliza nome de ativo para comparação."""    if pd.isna(nome):        return ""    nome = str(nome).lower().strip()    acentos = {'á':'a','à':'a','ã':'a','â':'a','é':'e','ê':'e','í':'i','ó':'o','ô':'o','õ':'o','ú':'u','ç':'c'}    for k, v in acentos.items():        nome = nome.replace(k, v)    nome = re.sub(r'[^a-z0-9\s]', ' ', nome)    nome = re.sub(r'\s+', ' ', nome).strip()    return nomedef similaridade(a: str, b: str) -> float:    """Retorna score de similaridade entre 0 e 1."""    return SequenceMatcher(None, normalizar_nome(a), normalizar_nome(b)).ratio()def parse_percentual(valor) -> float:    """Converte valor percentual para float decimal."""    if pd.isna(valor) or valor == '-' or valor == '':        return np.nan    if isinstance(valor, (int, float)):        if abs(valor) > 1:            return valor / 100        return valor    valor = str(valor).strip().replace('%', '').replace(',', '.')    try:        v = float(valor)        return v / 100    except:        return np.nandef ler_retornos_aba_excel(xlsx_path: str, sheet_name: str) -> pd.Series:    """Lê retornos mensais de uma aba do Excel."""    df = pd.read_excel(xlsx_path, sheet_name=sheet_name, header=None, engine='openpyxl')        header_row = None    for i, row in df.iterrows():        if str(row.iloc[0]).strip().lower() == 'ano':            header_row = i            break        if header_row is None:        raise ValueError(f"Não encontrei cabeçalho 'Ano' na aba '{sheet_name}'")        meses_map = {'jan':1,'fev':2,'mar':3,'abr':4,'mai':5,'jun':6,                 'jul':7,'ago':8,'set':9,'out':10,'nov':11,'dez':12}        header = [str(c).strip().lower()[:3] for c in df.iloc[header_row].values]        retornos = []    for i in range(header_row + 1, len(df)):        row = df.iloc[i]        ano_val = row.iloc[0]                if pd.isna(ano_val) or str(ano_val).strip().lower() in ['', 'estatísticas', 'estatisticas', 'meses']:            break                try:            ano = int(ano_val)        except:            break                for col_idx, col_name in enumerate(header[1:], start=1):            if col_name in meses_map:                mes = meses_map[col_name]                val = parse_percentual(row.iloc[col_idx])                if not pd.isna(val):                    data = pd.Timestamp(year=ano, month=mes, day=1)                    retornos.append({'data': data, 'retorno': val})        if not retornos:        return pd.Series(dtype=float)        df_ret = pd.DataFrame(retornos).set_index('data').sort_index()    return df_ret['retorno']def ler_pesos_resumo(xlsx_path: str) -> Dict[str, float]:    """Lê pesos da aba 'Resumo' do Excel."""    df = pd.read_excel(xlsx_path, sheet_name='Resumo', header=None, engine='openpyxl')        header_row = None    col_ativo = None    col_peso = None        for i, row in df.iterrows():        for j, val in enumerate(row):            val_str = str(val).strip().lower()            if val_str == 'ativo':                col_ativo = j            if 'peso' in val_str and 'cartei' in val_str:                col_peso = j        if col_ativo is not None and col_peso is not None:            header_row = i            break        if header_row is None:        return {}        pesos = {}    for i in range(header_row + 1, len(df)):        row = df.iloc[i]        ativo = row.iloc[col_ativo]        peso = row.iloc[col_peso]                if pd.isna(ativo) or str(ativo).strip() == '':            continue                ativo = str(ativo).strip()        try:            peso_val = float(peso) if not pd.isna(peso) else 0.0            if peso_val > 1:                peso_val = peso_val / 100            pesos[ativo] = peso_val        except:            pesos[ativo] = 0.0        return pesosdef carregar_dados_excel(xlsx_path: str, min_obs: int = 12):    """Carrega retornos mensais e pesos do Excel."""    xlsx = pd.ExcelFile(xlsx_path, engine='openpyxl')    abas = xlsx.sheet_names        relatorio = {        'total_abas': len(abas),        'abas_processadas': [],        'abas_excluidas': [],        'meses_por_ativo': {}    }        pesos_carteira = ler_pesos_resumo(xlsx_path)        series_list = {}    for aba in abas:        if aba.lower() == 'resumo':            continue                try:            serie = ler_retornos_aba_excel(xlsx_path, aba)            n_obs = len(serie.dropna())            relatorio['meses_por_ativo'][aba] = n_obs                        if n_obs >= min_obs:                series_list[aba] = serie                relatorio['abas_processadas'].append(aba)            else:                relatorio['abas_excluidas'].append((aba, f'apenas {n_obs} meses < {min_obs}'))        except Exception as e:            relatorio['abas_excluidas'].append((aba, str(e)[:50]))        if not series_list:        raise ValueError("Nenhum ativo com dados suficientes!")        retornos_df = pd.DataFrame(series_list)    return retornos_df, pesos_carteira, relatorioprint("✅ Funções de leitura carregadas!")

In [ ]:
# =============================================================================# FUNÇÕES - ESTATÍSTICAS E OTIMIZAÇÃO# =============================================================================def retorno_geometrico_anual(retornos_mensais: pd.Series) -> float:    """Calcula retorno geométrico anualizado."""    ret = retornos_mensais.dropna()    if len(ret) == 0:        return np.nan    prod = np.prod(1 + ret)    n = len(ret)    return float(prod ** (12 / n) - 1)def retorno_aritmetico_anual(retornos_mensais: pd.Series) -> float:    """Calcula retorno aritmético anualizado."""    return float(retornos_mensais.mean() * 12)def volatilidade_anual(retornos_mensais: pd.Series, vol_floor: float = 0.0) -> float:    """Calcula volatilidade anualizada."""    vol_mensal = retornos_mensais.std()    vol_mensal = max(vol_mensal, vol_floor)    return float(vol_mensal * np.sqrt(12))def calcular_estatisticas_portfolio(retornos_df, usar_geometrico=True, vol_floor=0.0):    """Calcula retornos esperados e matriz de covariância anualizados."""    if usar_geometrico:        ret_func = retorno_geometrico_anual    else:        ret_func = retorno_aritmetico_anual        retornos_esperados = retornos_df.apply(ret_func)    cov_matrix = retornos_df.cov() * 12        return retornos_esperados, cov_matrixdef corrigir_matriz_psd(cov_matrix, epsilon=1e-8):    """Verifica se matriz é PSD e corrige se necessário."""    arr = cov_matrix.values    eigenvalues, eigenvectors = eigh(arr)        if np.min(eigenvalues) >= -epsilon:        return cov_matrix, False        eigenvalues_corrigidos = np.maximum(eigenvalues, epsilon)    arr_corrigido = eigenvectors @ np.diag(eigenvalues_corrigidos) @ eigenvectors.T    arr_corrigido = (arr_corrigido + arr_corrigido.T) / 2        cov_corrigida = pd.DataFrame(arr_corrigido, index=cov_matrix.index, columns=cov_matrix.columns)    return cov_corrigida, Truedef calcular_metricas_portfolio(pesos, retornos_esperados, cov_matrix, rf_anual=0.0):    """Calcula métricas de um portfólio."""    pesos = np.array(pesos, dtype=float)    retorno = float(np.dot(pesos, retornos_esperados))    volatilidade = float(np.sqrt(np.dot(pesos.T, np.dot(cov_matrix, pesos))))        if volatilidade > 0:        sharpe = (retorno - rf_anual) / volatilidade    else:        sharpe = np.nan        return retorno, volatilidade, sharpedef otimizar_min_volatilidade(retornos_esperados, cov_matrix, peso_min=0.0, peso_max=1.0):    """Encontra carteira GMV."""    n = len(retornos_esperados)    pesos_iniciais = np.ones(n) / n    limites = tuple((peso_min, peso_max) for _ in range(n))    restricoes = {'type': 'eq', 'fun': lambda x: np.sum(x) - 1.0}        def objetivo(w):        return np.sqrt(np.dot(w.T, np.dot(cov_matrix, w)))        result = minimize(objetivo, pesos_iniciais, method='SLSQP', bounds=limites, constraints=restricoes)    if not result.success:        print(f"⚠️ Otimização Min Vol não convergiu: {result.message}")    return result.xdef otimizar_max_sharpe(retornos_esperados, cov_matrix, rf_anual=0.0, peso_min=0.0, peso_max=1.0):    """Encontra carteira de máximo Sharpe."""    n = len(retornos_esperados)    pesos_iniciais = np.ones(n) / n    limites = tuple((peso_min, peso_max) for _ in range(n))    restricoes = {'type': 'eq', 'fun': lambda x: np.sum(x) - 1.0}        def objetivo(w):        ret = np.dot(w, retornos_esperados)        vol = np.sqrt(np.dot(w.T, np.dot(cov_matrix, w)))        if vol == 0:            return 1e10        return -(ret - rf_anual) / vol        result = minimize(objetivo, pesos_iniciais, method='SLSQP', bounds=limites, constraints=restricoes)    if not result.success:        print(f"⚠️ Otimização Max Sharpe não convergiu: {result.message}")    return result.xdef simular_portfolios_monte_carlo(retornos_esperados, cov_matrix, rf_anual, n_portfolios=10000, seed=42):    """Simula carteiras aleatórias."""    np.random.seed(seed)    n_ativos = len(retornos_esperados)        n_conc = int(n_portfolios * 0.6)    n_div = n_portfolios - n_conc        w_conc = np.random.dirichlet([0.25] * n_ativos, size=n_conc)    w_div = np.random.dirichlet([1.0] * n_ativos, size=n_div)    pesos = np.vstack([w_conc, w_div])        retornos = pesos @ retornos_esperados    volatilidades = np.sqrt(np.einsum('ij,jk,ik->i', pesos, cov_matrix, pesos))    sharpes = np.where(volatilidades > 0, (retornos - rf_anual) / volatilidades, np.nan)        return retornos, volatilidades, sharpes, pesosprint("✅ Funções de estatística e otimização carregadas!")

In [ ]:
# =============================================================================# FUNÇÕES - TAXA LIVRE DE RISCO (CDI)# =============================================================================def obter_cdi_atual():    """Obtém CDI atual via API do BCB."""    try:        url = "https://api.bcb.gov.br/dados/serie/bcdata.sgs.12/dados/ultimos/1?formato=json"        df = pd.read_json(url)        if df.empty:            raise ValueError("SGS retornou vazio")        valor_str = str(df.loc[0, "valor"]).replace(",", ".")        taxa_diaria = float(valor_str) / 100.0        return float((1.0 + taxa_diaria) ** 252 - 1.0)    except Exception as e:        print(f"⚠️ Erro ao obter CDI: {e}")        return Nonedef obter_taxa_livre_risco(modo, fallback, manual=None, data_inicio=None, data_fim=None):    """Obtém taxa livre de risco conforme modo especificado."""    if modo == "manual":        return manual if manual is not None else fallback        if modo == "atual":        rf = obter_cdi_atual()        return rf if rf is not None else fallback        return fallbackprint("✅ Funções de RF carregadas!")

In [ ]:
# =============================================================================# FUNÇÕES - VISUALIZAÇÃO (PLOTLY)# =============================================================================def criar_grafico_fronteira(ret_sim, vol_sim, sharpe_sim, carteiras_especiais, rf_anual, titulo="Fronteira Eficiente"):    """Cria gráfico interativo da fronteira eficiente."""    fig = go.Figure()        # Nuvem de portfólios    fig.add_trace(go.Scatter(        x=vol_sim * 100, y=ret_sim * 100,        mode='markers',        marker=dict(size=3, color=sharpe_sim, colorscale='Viridis',                    colorbar=dict(title='Sharpe', x=1.02), opacity=0.5),        name='Portfólios Simulados',        hovertemplate='Vol: %{x:.2f}%<br>Ret: %{y:.2f}%<br>Sharpe: %{marker.color:.3f}<extra></extra>'    ))        # Estilos para carteiras especiais    estilos = {        'GMV': {'color': 'blue', 'symbol': 'diamond', 'size': 18},        'Max Sharpe': {'color': 'green', 'symbol': 'triangle-up', 'size': 18},        'Carteira Atual': {'color': 'gold', 'symbol': 'star', 'size': 24}    }        for nome, dados in carteiras_especiais.items():        estilo = estilos.get(nome, {'color': 'red', 'symbol': 'circle', 'size': 15})                pesos = dados.get('pesos', [])        nomes = dados.get('nomes', [])        if len(pesos) > 0 and len(nomes) > 0:            idx_sorted = np.argsort(pesos)[::-1]            top_pesos = '<br>'.join([f"{nomes[i]}: {pesos[i]*100:.1f}%" for i in idx_sorted[:10] if pesos[i] > 0.01])        else:            top_pesos = ''                hover_text = (f"<b>{nome}</b><br>"                      f"Retorno: {dados['retorno']*100:.2f}%<br>"                      f"Volatilidade: {dados['volatilidade']*100:.2f}%<br>"                      f"Sharpe: {dados['sharpe']:.3f}<br><br>"                      f"<b>Top Pesos:</b><br>{top_pesos}")                fig.add_trace(go.Scatter(            x=[dados['volatilidade'] * 100], y=[dados['retorno'] * 100],            mode='markers+text',            marker=dict(size=estilo['size'], color=estilo['color'], symbol=estilo['symbol'],                        line=dict(width=2, color='black')),            text=[nome], textposition='top center', name=nome,            hovertemplate=hover_text + '<extra></extra>'        ))        fig.add_hline(y=rf_anual * 100, line_dash="dash", line_color="gray",                  annotation_text=f"RF (CDI): {rf_anual*100:.2f}%")        fig.update_layout(        title=dict(text=titulo, x=0.5),        xaxis_title="Volatilidade Anual (%)",        yaxis_title="Retorno Anual Esperado (%)",        template="plotly_white", width=1000, height=700,        legend=dict(x=0.02, y=0.98), hovermode='closest'    )        return figdef criar_heatmap_correlacao(corr_matrix, titulo="Matriz de Correlação"):    """Cria heatmap interativo da matriz de correlação."""    fig = go.Figure(data=go.Heatmap(        z=corr_matrix.values,        x=list(corr_matrix.columns), y=list(corr_matrix.index),        colorscale='RdBu', zmid=0, zmin=-1, zmax=1,        colorbar=dict(title='Correlação'),        hovertemplate='%{x} vs %{y}<br>Corr: %{z:.3f}<extra></extra>'    ))        fig.update_layout(title=dict(text=titulo, x=0.5), template="plotly_white",                      width=800, height=700, xaxis=dict(side='top'))    return figprint("✅ Funções de visualização carregadas!")

---## 📥 BLOCO 2 - CARREGAMENTO DE DADOSImplementa a hierarquia de fontes: Excel → yfinance → DEFAULT_UNIVERSE

In [ ]:
# =============================================================================# DECISÃO: QUAL FONTE DE DADOS USAR?# =============================================================================USAR_EXCEL = Trueexcel_existe = Path(EXCEL_PATH).exists()print("=" * 60)print("SELEÇÃO DE FONTE DE DADOS")print("=" * 60)if USAR_EXCEL and excel_existe:    FONTE_DADOS = "EXCEL"    print(f"✅ Usando EXCEL: {EXCEL_PATH}")elif USAR_EXCEL and not excel_existe:    print(f"⚠️ Excel não encontrado em: {EXCEL_PATH}")    if YFINANCE_DISPONIVEL:        FONTE_DADOS = "YFINANCE_DEFAULT"        print(f"   Usando DEFAULT_UNIVERSE via yfinance")    else:        raise FileNotFoundError("Excel não encontrado e yfinance não disponível!")else:    FONTE_DADOS = "YFINANCE_DEFAULT"print(f"\n📊 Fonte selecionada: {FONTE_DADOS}")

In [ ]:
# =============================================================================# CARREGAMENTO DOS DADOS# =============================================================================if FONTE_DADOS == "EXCEL":    print("\n📂 Carregando dados do Excel...")    print("-" * 50)        retornos_df, pesos_carteira_raw, relatorio = carregar_dados_excel(EXCEL_PATH, min_obs=MIN_OBS)        print(f"\n✅ Ativos carregados: {len(relatorio['abas_processadas'])}")        if relatorio['abas_excluidas']:        print(f"\n⚠️ Ativos excluídos ({len(relatorio['abas_excluidas'])})")        print(f"\n📅 Período: {retornos_df.index.min().strftime('%Y-%m')} a {retornos_df.index.max().strftime('%Y-%m')}")    print(f"   Total de meses: {len(retornos_df)}")else:    raise NotImplementedError("Fonte yfinance não implementada nesta versão.")ATIVOS_OTIMIZACAO = list(retornos_df.columns)NUM_ATIVOS = len(ATIVOS_OTIMIZACAO)print(f"\n🎯 Ativos no universo de otimização: {NUM_ATIVOS}")

In [ ]:
# =============================================================================# MAPEAMENTO DE PESOS DA CARTEIRA ATUAL# =============================================================================if FONTE_DADOS == "EXCEL" and pesos_carteira_raw:    print("\n" + "=" * 60)    print("MAPEAMENTO DE PESOS DA CARTEIRA ATUAL")    print("=" * 60)        mapeamento = {}    nao_mapeados = []        for ativo_resumo, peso in pesos_carteira_raw.items():        if peso == 0:            continue                if ativo_resumo in ATIVOS_OTIMIZACAO:            mapeamento[ativo_resumo] = {'aba': ativo_resumo, 'peso': peso, 'score': 1.0}            continue                melhor_match = None        melhor_score = 0        for aba in ATIVOS_OTIMIZACAO:            score = similaridade(ativo_resumo, aba)            if score > melhor_score:                melhor_score = score                melhor_match = aba                if melhor_score >= 0.6:            mapeamento[ativo_resumo] = {'aba': melhor_match, 'peso': peso, 'score': melhor_score}        else:            nao_mapeados.append((ativo_resumo, peso, melhor_match, melhor_score))        print(f"\n✅ Ativos mapeados: {len(mapeamento)}")        if nao_mapeados:        peso_nao_mapeado = sum(x[1] for x in nao_mapeados)        print(f"\n⚠️ Ativos NÃO mapeados: {len(nao_mapeados)} (peso: {peso_nao_mapeado:.2%})")        # Criar vetor de pesos    pesos_carteira = np.zeros(NUM_ATIVOS)    for ativo, info in mapeamento.items():        try:            idx = ATIVOS_OTIMIZACAO.index(info['aba'])            pesos_carteira[idx] += info['peso']        except ValueError:            pass        soma_pesos = pesos_carteira.sum()    print(f"\n📊 Soma dos pesos mapeados: {soma_pesos:.2%}")        if soma_pesos > 0 and soma_pesos != 1.0:        print(f"   Renormalizando para 100%")        pesos_carteira = pesos_carteira / soma_pesos        TEM_CARTEIRA_ATUAL = soma_pesos > 0else:    pesos_carteira = None    TEM_CARTEIRA_ATUAL = False    print("\n⚠️ Pesos da carteira atual não disponíveis.")

---## 📈 BLOCO 3 - CÁLCULO DE ESTATÍSTICAS

In [ ]:
# =============================================================================# TAXA LIVRE DE RISCO# =============================================================================print("\n" + "=" * 60)print("TAXA LIVRE DE RISCO (CDI)")print("=" * 60)RF_ANUAL = obter_taxa_livre_risco(RF_MODO, RF_FALLBACK, RF_MANUAL, DATA_INICIO, DATA_FIM)print(f"\n📊 RF anual (CDI): {RF_ANUAL*100:.2f}% a.a.")print(f"   Modo: {RF_MODO}")

In [ ]:
# =============================================================================# ESTATÍSTICAS DOS ATIVOS# =============================================================================print("\n" + "=" * 60)print("ESTATÍSTICAS DOS ATIVOS")print("=" * 60)retornos_esperados, cov_matrix = calcular_estatisticas_portfolio(    retornos_df, usar_geometrico=USAR_MEDIA_GEOMETRICA, vol_floor=VOL_FLOOR)cov_matrix, foi_corrigida = corrigir_matriz_psd(cov_matrix)if foi_corrigida:    print("\n⚠️ Matriz de covariância corrigida para PSD")corr_matrix = retornos_df.corr()volatilidades = retornos_df.apply(lambda x: volatilidade_anual(x, VOL_FLOOR))print(f"\n{'Ativo':<25} {'Ret. Anual':<15} {'Vol. Anual':<15} {'Sharpe':<10}")print("-" * 65)for ativo in ATIVOS_OTIMIZACAO[:15]:    ret = retornos_esperados[ativo]    vol = volatilidades[ativo]    sharpe = (ret - RF_ANUAL) / vol if vol > 0 else np.nan    print(f"{ativo:<25} {ret*100:>12.2f}% {vol*100:>12.2f}% {sharpe:>10.2f}")if len(ATIVOS_OTIMIZACAO) > 15:    print(f"... e mais {len(ATIVOS_OTIMIZACAO) - 15} ativos")

---## ⚙️ BLOCO 4 - OTIMIZAÇÃO DE PORTFÓLIO

In [ ]:
# =============================================================================# OTIMIZAÇÃO: GMV E MÁXIMO SHARPE# =============================================================================print("\n" + "=" * 60)print("OTIMIZAÇÃO DE PORTFÓLIO")print("=" * 60)ret_array = retornos_esperados.valuescov_array = cov_matrix.valuesprint("\n🔍 Otimizando GMV...")pesos_gmv = otimizar_min_volatilidade(ret_array, cov_array, PESO_MIN_ATIVO, PESO_MAX_ATIVO)ret_gmv, vol_gmv, sharpe_gmv = calcular_metricas_portfolio(pesos_gmv, ret_array, cov_array, RF_ANUAL)print("🔍 Otimizando Máximo Sharpe...")pesos_max_sharpe = otimizar_max_sharpe(ret_array, cov_array, RF_ANUAL, PESO_MIN_ATIVO, PESO_MAX_ATIVO)ret_max_sharpe, vol_max_sharpe, sharpe_max_sharpe = calcular_metricas_portfolio(pesos_max_sharpe, ret_array, cov_array, RF_ANUAL)if TEM_CARTEIRA_ATUAL:    ret_atual, vol_atual, sharpe_atual = calcular_metricas_portfolio(pesos_carteira, ret_array, cov_array, RF_ANUAL)print("\n" + "=" * 60)print("RESULTADOS")print("=" * 60)print(f"\n{'Carteira':<20} {'Retorno':<15} {'Volatilidade':<15} {'Sharpe':<10}")print("-" * 60)print(f"{'GMV':<20} {ret_gmv*100:>12.2f}% {vol_gmv*100:>12.2f}% {sharpe_gmv:>10.3f}")print(f"{'Max Sharpe':<20} {ret_max_sharpe*100:>12.2f}% {vol_max_sharpe*100:>12.2f}% {sharpe_max_sharpe:>10.3f}")if TEM_CARTEIRA_ATUAL:    print(f"{'Carteira Atual ⭐':<20} {ret_atual*100:>12.2f}% {vol_atual*100:>12.2f}% {sharpe_atual:>10.3f}")

---## 🎲 BLOCO 5 - SIMULAÇÃO MONTE CARLO

In [ ]:
# =============================================================================# SIMULAÇÃO MONTE CARLO# =============================================================================print("\n" + "=" * 60)print(f"SIMULAÇÃO MONTE CARLO ({NUM_PORTFOLIOS:,} portfólios)")print("=" * 60)ret_sim, vol_sim, sharpe_sim, pesos_sim = simular_portfolios_monte_carlo(    ret_array, cov_array, RF_ANUAL, NUM_PORTFOLIOS, RANDOM_SEED)print(f"\n✅ Simulação concluída!")print(f"   Retorno: min={ret_sim.min()*100:.2f}%, max={ret_sim.max()*100:.2f}%")print(f"   Volatilidade: min={vol_sim.min()*100:.2f}%, max={vol_sim.max()*100:.2f}%")

---## 📊 BLOCO 6 - VISUALIZAÇÕES

In [ ]:
# =============================================================================# GRÁFICO: FRONTEIRA EFICIENTE# =============================================================================carteiras_especiais = {    'GMV': {        'retorno': ret_gmv, 'volatilidade': vol_gmv, 'sharpe': sharpe_gmv,        'pesos': pesos_gmv, 'nomes': ATIVOS_OTIMIZACAO    },    'Max Sharpe': {        'retorno': ret_max_sharpe, 'volatilidade': vol_max_sharpe, 'sharpe': sharpe_max_sharpe,        'pesos': pesos_max_sharpe, 'nomes': ATIVOS_OTIMIZACAO    }}if TEM_CARTEIRA_ATUAL:    carteiras_especiais['Carteira Atual'] = {        'retorno': ret_atual, 'volatilidade': vol_atual, 'sharpe': sharpe_atual,        'pesos': pesos_carteira, 'nomes': ATIVOS_OTIMIZACAO    }fig_frontier = criar_grafico_fronteira(    ret_sim, vol_sim, sharpe_sim, carteiras_especiais, RF_ANUAL,    titulo="Fronteira Eficiente - PyPortfolio v2.3")fig_frontier.show()

In [ ]:
# =============================================================================# GRÁFICO: MATRIZ DE CORRELAÇÃO# =============================================================================fig_corr = criar_heatmap_correlacao(corr_matrix, "Matriz de Correlação - Retornos Mensais")fig_corr.show()

---## 📋 BLOCO 7 - RESUMO FINAL

In [ ]:
# =============================================================================# RESUMO FINAL# =============================================================================print("\n" + "=" * 70)print("RESUMO DA ANÁLISE - PyPortfolio v2.3")print("=" * 70)print(f"\n📊 CONFIGURAÇÃO")print(f"   Fonte de dados: {FONTE_DADOS}")print(f"   Período: {retornos_df.index.min().strftime('%Y-%m')} a {retornos_df.index.max().strftime('%Y-%m')}")print(f"   Ativos no universo: {NUM_ATIVOS}")print(f"   MIN_OBS: {MIN_OBS} meses | Média: {'Geométrica' if USAR_MEDIA_GEOMETRICA else 'Aritmética'}")print(f"   Taxa livre de risco (CDI): {RF_ANUAL*100:.2f}% a.a.")print(f"\n📈 CARTEIRAS OTIMIZADAS")print(f"   {'Carteira':<20} {'Retorno':<12} {'Vol':<12} {'Sharpe':<10}")print(f"   {'-'*54}")print(f"   {'GMV':<20} {ret_gmv*100:>9.2f}%  {vol_gmv*100:>9.2f}%  {sharpe_gmv:>8.3f}")print(f"   {'Max Sharpe':<20} {ret_max_sharpe*100:>9.2f}%  {vol_max_sharpe*100:>9.2f}%  {sharpe_max_sharpe:>8.3f}")if TEM_CARTEIRA_ATUAL:    print(f"   {'Carteira Atual ⭐':<20} {ret_atual*100:>9.2f}%  {vol_atual*100:>9.2f}%  {sharpe_atual:>8.3f}")if TEM_CARTEIRA_ATUAL:    print(f"\n💡 DIAGNÓSTICO DA CARTEIRA ATUAL")    diff_sharpe = sharpe_max_sharpe - sharpe_atual    diff_vol = vol_atual - vol_gmv        if diff_sharpe > 0.1:        print(f"   ⚠️  Sharpe atual está {diff_sharpe:.2f} abaixo do ótimo")    else:        print(f"   ✅ Sharpe atual está próximo do ótimo")        if diff_vol > 0.02:        print(f"   ⚠️  Volatilidade {diff_vol*100:.1f}pp acima do GMV")print("\n" + "=" * 70)print("✅ Análise concluída com sucesso!")print("=" * 70)

---## 📝 Notas e Próximos Passos### Limitações conhecidas:- Retornos passados não garantem retornos futuros- Matriz de covariância estimada pode não refletir correlações futuras- Ativos de renda fixa com volatilidade muito baixa podem distorcer a otimização### Sugestões de melhoria:1. Implementar Black-Litterman para incorporar views do investidor2. Adicionar restrições por classe de ativo3. Incluir análise de contribuição de risco (Risk Parity)4. Backtesting com janela móvel5. Penalidade de turnover vs carteira atual